In [20]:
#!/usr/bin/env python3

import numpy as np
import pandas as pd
import xarray as xr
import zipfile
import glob
import netCDF4 as nc
import xesmf as xe
from scipy import stats

def trend_finder(timeseries_maps):
    """
    Find decadal trends at each lat and lon point. 
    """
    shape_of_maps = np.shape(timeseries_maps)
    # must be adjusted based on len of time used for trends
    years = np.linspace(0,3.5,36)
    trend_map = []
    for lat in range(0, shape_of_maps[1]):
        lat_line = []
        for lon in range(0, shape_of_maps[2]):
            timeseries = timeseries_maps[:,lat,lon]
            trend = stats.linregress(x=years, y=timeseries)[0] # in K/dec
            lat_line.append(trend)
        trend_map.append(lat_line)
    return(np.array(trend_map))
            
# load path to CMIP 6 data
path_to_CMIP6_data = '/home/disk/pna2/aodhan/CMIP6/historical_monthly_psl_google/*'
model_paths = glob.glob(path_to_CMIP6_data)

# here we select the start times of trends, this can be changed
start_times = np.arange(1854,1980,5)

# for each model...
for model in model_paths[11:-1]: # the -1 is because the last "model" is the directory /trendmap where data is stored
    realizations = glob.glob(model + '/*')

    # create data dictionary
    model_trends = []
    
    # for each realization...
    simulation_index = []
    simulation_count = 1
    for ensemble_member in realizations:

        # if we run into problems with given simulation, skip it
        try:
            simulation = xr.open_dataset(ensemble_member)
            model_name = simulation.source_id
            simulation_name = simulation.variant_label
            simulation_index.append(simulation_count)

            # CMIP6 models must be regridded, below we define input and output grids
            latitudes = simulation.lat.values
            longitudes = simulation.lon.values
            InputGrid = {"lon": longitudes, "lat": latitudes}
            OutputGrid = {"lon": np.arange(1.25, 358.751, 2.5), "lat": np.arange(-88.75, 88.751, 2.5)}
            regridder = xe.Regridder(InputGrid, OutputGrid, "bilinear")
            
            # create lists for where to append data
            all_timeperiod_trends = []
            time_keys = []
            
            # find time slices for just winter
            for i in start_times:
                start_date = str(i) + '-01'
                end_date = str(i+35) + '-12'

                # find correct time period of data
                time_slice_data = simulation.sel(time=slice(start_date, end_date))
                data_array = time_slice_data.tas.values

                # find trends for each of the time periods
                shape_of_data_array = np.shape(data_array)
                data_calendar = np.reshape(data_array, (36, 12, shape_of_data_array[1], shape_of_data_array[2]))
                season_calendar = [data_calendar[:,10],data_calendar[:,11], data_calendar[:,0], 
                                data_calendar[:,1], data_calendar[:,2]]
                timeseries_map = np.nanmean(season_calendar, axis=0)
                trend_map = trend_finder(timeseries_map)
                trend_map_2p5x2p5 = regridder(trend_map)
                all_timeperiod_trends.append(trend_map_2p5x2p5)
                time_keys.append(i)

            # Increase simulation count index
            simulation_count += 1
            print(model_name, simulation_name)

            # append all timeperiod trends 
            model_trends.append(all_timeperiod_trends)
        except:
            print('Skipping: ', model_name, simulation_name)
            continue
    EnsembleTrendsArray = np.array(model_trends)
    ForcedTrend = np.nanmean(EnsembleTrendsArray, axis=0)
    NaturalTrendsArray = EnsembleTrendsArray - ForcedTrend
    ForcedNaturalArray = [[NaturalTrendsArray[i], ForcedTrend] for i in range(0, len(NaturalTrendsArray))]

    # Timeperiod data will be dumped into NetCDF files
    fileName = path_to_CMIP6_data[:-2] + '/trendmap/' + model_name.replace("-", "_") + '_PSL_NDJFM_TrendMaps.nc'

    # Create netcdf file with dimensions
    ds = nc.Dataset(fileName, 'w', format='NETCDF4')
    ensemble_member = ds.createDimension('ensemble_member', len(simulation_index))
    ForcedNatural = ds.createDimension('ForcedNatural', 2)
    TrendTimePeriod = ds.createDimension('TrendTimePeriod', len(time_keys)) # 26 timeperiods
    Lat = ds.createDimension('Lat', 72)
    Lon = ds.createDimension('Lon', 144)

    # Add variables to dimensions
    ensemble_member = ds.createVariable('ensemble_member', int, ('ensemble_member',))
    ForcedNatural = ds.createVariable('ForcedNatural', int, ('ForcedNatural',))
    TrendTimePeriod = ds.createVariable('TrendTimePeriod', int, ('TrendTimePeriod',))
    Lat = ds.createVariable('Lat', 'f4', ('Lat',))
    Lon = ds.createVariable('Lon', 'f4', ('Lon',))
    Ts_trends = ds.createVariable('ts_trend', 'f4', ('ensemble_member', 'ForcedNatural', 'TrendTimePeriod', 'Lat', 'Lon'))

    # Assing values to variables
    ensemble_member[:] = simulation_index
    ForcedNatural[:] = [0,1]
    TrendTimePeriod[:] = time_keys
    Lat[:] = np.arange(-88.75, 88.751, 2.5)
    Lon[:] = np.arange(1.25, 358.751, 2.5)
    Ts_trends[:] = ForcedNaturalArray

    ds.close()


ACCESS-ESM1-5 r1i1p1f1
ACCESS-ESM1-5 r2i1p1f1
ACCESS-ESM1-5 r3i1p1f1
ACCESS-ESM1-5 r9i1p1f1
ACCESS-ESM1-5 r4i1p1f1
ACCESS-ESM1-5 r6i1p1f1
ACCESS-ESM1-5 r8i1p1f1
ACCESS-ESM1-5 r7i1p1f1
ACCESS-ESM1-5 r5i1p1f1
ACCESS-ESM1-5 r10i1p1f1
ACCESS-ESM1-5 r19i1p1f1
ACCESS-ESM1-5 r18i1p1f1
ACCESS-ESM1-5 r20i1p1f1
ACCESS-ESM1-5 r17i1p1f1
ACCESS-ESM1-5 r16i1p1f1
ACCESS-ESM1-5 r15i1p1f1
ACCESS-ESM1-5 r14i1p1f1
ACCESS-ESM1-5 r13i1p1f1
ACCESS-ESM1-5 r11i1p1f1
ACCESS-ESM1-5 r12i1p1f1
ACCESS-ESM1-5 r28i1p1f1
ACCESS-ESM1-5 r29i1p1f1
ACCESS-ESM1-5 r30i1p1f1
ACCESS-ESM1-5 r25i1p1f1
ACCESS-ESM1-5 r26i1p1f1
ACCESS-ESM1-5 r24i1p1f1
ACCESS-ESM1-5 r22i1p1f1
ACCESS-ESM1-5 r21i1p1f1
ACCESS-ESM1-5 r23i1p1f1
ACCESS-ESM1-5 r27i1p1f1
ACCESS-ESM1-5 r38i1p1f1
ACCESS-ESM1-5 r33i1p1f1
ACCESS-ESM1-5 r32i1p1f1
ACCESS-ESM1-5 r31i1p1f1
ACCESS-ESM1-5 r36i1p1f1
ACCESS-ESM1-5 r35i1p1f1
ACCESS-ESM1-5 r40i1p1f1
ACCESS-ESM1-5 r37i1p1f1
ACCESS-ESM1-5 r39i1p1f1
ACCESS-ESM1-5 r34i1p1f1
EC-Earth3 r9i1p1f1
EC-Earth3 r11i1p1f1
EC-Earth3 

ValueError: shape mismatch: objects cannot be broadcast to a single shape

In [6]:
#!/usr/bin/env python3

import numpy as np
import pandas as pd
import xarray as xr
import zipfile
import glob
import netCDF4 as nc
import xesmf as xe
from scipy import stats

def trend_finder(timeseries_maps):
    """
    Find decadal trends at each lat and lon point. 
    """
    shape_of_maps = np.shape(timeseries_maps)
    # must be adjusted based on len of time used for trends
    years = np.linspace(0,3.5,36)
    trend_map = []
    for lat in range(0, shape_of_maps[1]):
        lat_line = []
        for lon in range(0, shape_of_maps[2]):
            timeseries = timeseries_maps[:,lat,lon]
            trend = stats.linregress(x=years, y=timeseries)[0] # in K/dec
            lat_line.append(trend)
        trend_map.append(lat_line)
    return(np.array(trend_map))
            
# load path to CMIP 6 data
path_to_CMIP6_data = '/home/disk/pna2/aodhan/CMIP6/historical_monthly_tas_google/*'
model_paths = glob.glob(path_to_CMIP6_data)

# here we select the start times of trends, this can be changed
start_times = np.arange(1854,1980,5)

# for each model...
for model in model_paths:
    realizations = glob.glob(model + '/*')

    # create data dictionary
    model_trends = []
    
    # for each realization...
    simulation_index = []
    for ensemble_member in realizations:
        simulation = xr.open_dataset(ensemble_member)
        model_name = simulation.source_id
        simulation_name = simulation.variant_label
        simulation_number = int(simulation_name.split('i')[0][1:])
        simulation_index.append(simulation_number)
        print(model_name, simulation_name, simulation_number)

         # CMIP6 models must be regridded, below we define input and output grids
        latitudes = simulation.lat.values
        longitudes = simulation.lon.values
        InputGrid = {"lon": longitudes, "lat": latitudes}
        OutputGrid = {"lon": np.arange(1.25, 358.751, 2.5), "lat": np.arange(-88.75, 88.751, 2.5)}
        regridder = xe.Regridder(InputGrid, OutputGrid, "bilinear")
        
        # create lists for where to append data
        all_timeperiod_trends = []
        time_keys = []

        # find time slices for just winter
        for i in start_times:
            start_date = str(i) + '-01'
            end_date = str(i+35) + '-12'

            # find correct time period of data
            time_slice_data = simulation.sel(time=slice(start_date, end_date))
            data_array = time_slice_data.psl.values

            # find trends for each of the time periods
            shape_of_data_array = np.shape(data_array)
            data_calendar = np.reshape(data_array, (36, 12, shape_of_data_array[1], shape_of_data_array[2]))
            season_calendar = [data_calendar[:,10],data_calendar[:,11], data_calendar[:,0], 
                               data_calendar[:,1], data_calendar[:,2]]
            timeseries_map = np.nanmean(season_calendar, axis=0)
            trend_map = trend_finder(timeseries_map)
            trend_map_2p5x2p5 = regridder(trend_map)
            all_timeperiod_trends.append(trend_map_2p5x2p5)
            time_keys.append(i)
        
        # append all timeperiod trends 
        model_trends.append(all_timeperiod_trends)

    # Timeperiod data will be dumped into NetCDF files
    fileName = path_to_CMIP6_data[:-2] + '/trendmap/' + model_name.replace("-", "_") + '_TAS_NDJFM_TrendMaps.nc'

    # Create netcdf file with dimensions
    ds = nc.Dataset(fileName, 'w', format='NETCDF4')
    ensemble_member = ds.createDimension('ensemble_member', len(simulation_index))
    TrendTimePeriod = ds.createDimension('TrendTimePeriod', len(time_keys)) # 26 timeperiods
    Lat = ds.createDimension('Lat', 72)
    Lon = ds.createDimension('Lon', 144)

    # Add variables to dimensions
    ensemble_member = ds.createVariable('ensemble_member', int, ('ensemble_member',))
    TrendTimePeriod = ds.createVariable('TrendTimePeriod', int, ('TrendTimePeriod',))
    Lat = ds.createVariable('Lat', 'f4', ('Lat',))
    Lon = ds.createVariable('Lon', 'f4', ('Lon',))
    Ts_trends = ds.createVariable('ts_trend', 'f4', ('ensemble_member', 'TrendTimePeriod', 'Lat', 'Lon'))

    # Assing values to variables
    ensemble_member[:] = simulation_index
    TrendTimePeriod[:] = time_keys
    Lat[:] = np.arange(-88.75, 88.751, 2.5)
    Lon[:] = np.arange(1.25, 358.751, 2.5)
    Ts_trends[:] = model_trends

    ds.close()


IPSL-CM6A-LR r8i1p1f1 8
IPSL-CM6A-LR r2i1p1f1 2
IPSL-CM6A-LR r30i1p1f1 30
IPSL-CM6A-LR r31i1p1f1 31
IPSL-CM6A-LR r3i1p1f1 3
IPSL-CM6A-LR r6i1p1f1 6
IPSL-CM6A-LR r27i1p1f1 27
IPSL-CM6A-LR r7i1p1f1 7
IPSL-CM6A-LR r9i1p1f1 9
IPSL-CM6A-LR r26i1p1f1 26
IPSL-CM6A-LR r21i1p1f1 21
IPSL-CM6A-LR r25i1p1f1 25
IPSL-CM6A-LR r20i1p1f1 20
IPSL-CM6A-LR r24i1p1f1 24
IPSL-CM6A-LR r29i1p1f1 29
IPSL-CM6A-LR r23i1p1f1 23
IPSL-CM6A-LR r22i1p1f1 22
IPSL-CM6A-LR r19i1p1f1 19
IPSL-CM6A-LR r18i1p1f1 18
IPSL-CM6A-LR r11i1p1f1 11
IPSL-CM6A-LR r12i1p1f1 12
IPSL-CM6A-LR r16i1p1f1 16
IPSL-CM6A-LR r17i1p1f1 17
IPSL-CM6A-LR r1i1p1f1 1
IPSL-CM6A-LR r5i1p1f1 5
IPSL-CM6A-LR r4i1p1f1 4
IPSL-CM6A-LR r28i1p1f1 28
IPSL-CM6A-LR r10i1p1f1 10
IPSL-CM6A-LR r13i1p1f1 13
IPSL-CM6A-LR r14i1p1f1 14
IPSL-CM6A-LR r15i1p1f1 15
IPSL-CM6A-LR r32i1p1f1 32
GISS-E2-1-G r2i1p1f1 2
GISS-E2-1-G r1i1p1f1 1
GISS-E2-1-G r5i1p1f1 5
GISS-E2-1-G r4i1p1f1 4
GISS-E2-1-G r3i1p1f1 3
GISS-E2-1-G r7i1p1f1 7
GISS-E2-1-G r6i1p1f1 6
GISS-E2-1-G r10i1p1f1 10


ValueError: cannot reshape array of size 0 into shape (36,12,256,512)